In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns 
sns.set(style='white', palette='muted', color_codes=True)
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import plotly.express as px 

In [ ]:
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor 
from sklearn.linear_model import LogisticRegression, LinearRegression 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler 
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

In [ ]:
data = pd.read_csv(r'c:\Github\Fullstack-Data-Analyst\Learning\the_data\data-lab-4-heart.csv')
# data

In [ ]:
# data.info()
data.duplicated().sum()
data.drop_duplicates(subset=None, keep='first', ignore_index=False, inplace=True)
data.duplicated().sum()

In [ ]:
mis_col = data.isnull().sum() * 100 /len(data)
# mis_col.sort_values(ascending=False)

In [ ]:
# data.describe()

In [ ]:
f,ax = plt.subplots(figsize=(10,8))
ax.set_title('Chol Outliers')

sns.boxplot(data.chol, color='c', ax=ax)

In [ ]:
data.corr()['target'].sort_values(ascending=False)

In [ ]:
f,ax = plt.subplots(figsize=(10,8))
ax.set_title('Correlation Table')

data_corr = data.corr()
sns.heatmap(data_corr, mask=np.zeros_like(data_corr, dtype=np.bool_), cmap=sns.diverging_palette(240,10, as_cmap=True),
square=True, ax=ax)

In [ ]:
y = data.target
x = data.drop(columns=['target'], axis=1)


In [ ]:
num_val = ['age','trestbps','chol','thalach','oldpeak' ]
cat_val = ['sex', 'cp', 'restecg', 'exang', 'slope', 'ca', 'thal']

sex_val = data.sex

In [ ]:
# transfo = [('num', StandardScaler(), num_val),('cat', OrdinalEncoder(), cat_val)]
# transform = ColumnTransformer(transformers=transfo)

In [ ]:
enco_data = [('num', StandardScaler(),num_val),('cat', OrdinalEncoder(),cat_val)]
transformed_data = ColumnTransformer(transformers=enco_data)
transformed_data
x = transformed_data.fit_transform(x)



In [ ]:
# col_trans = ColumnTransformer([ 
#     ('num', StandardScaler(), num_val),
#     ('cat', OrdinalEncoder(), cat_val)
# ])

# x = pd.DataFrame(col_trans.fit_transform(x))
# x

In [ ]:
# ord_sex = data.sex 

# orig_dic_sex = np.unique(sex_val)
# ord_dic_sex = np.unique(ord_sex)

# sex_dictionary = dict(zip(orig_dic_sex,ord_dic_sex))
# sex_dictionary

In [ ]:
# f,ax = plt.subplots(figsize=(10,8))
# ax.set_title('Target Sex distribution')
# sns.histplot(data[(data.sex==1)]['target'],color='c',ax=ax)

In [ ]:
data.head(1)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.3, random_state=42)

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train,y_train)

In [ ]:
pred_logreg_train = logreg.predict(x_train)
pred_logreg_test = logreg.predict(x_test)

In [ ]:
score_acc_logreg_train = accuracy_score(y_train, pred_logreg_train).round(3)
score_acc_logreg_test = accuracy_score(y_test, pred_logreg_test).round(3)

acu_perf = pd.DataFrame([ 
    'LogisticRegression', score_acc_logreg_train, score_acc_logreg_test
]).transpose()
acu_perf.columns = ['Method', 'Accuracy Train Score', 'Accuracy Test Score']
acu_perf

In [ ]:
#Confusion Matrix are for classifiers and work same as any matrix

train_conmat = { 
    'y_train' : y_train,
    'pred_logreg_train': pred_logreg_train
}

data_matrix = pd.DataFrame( 
    train_conmat, columns=['y_train', 'pred_logreg_train']
)
confusion_matrix = pd.crosstab(data_matrix['y_train'], data_matrix['pred_logreg_train'], rownames=['True Train Values'], colnames=['Predicted Values'])

sns.heatmap(confusion_matrix, annot=True)
plt.show()

In [ ]:

test_data = { 
    'y_test': y_test,
    'pred_logreg_test' : pred_logreg_test
}

con_matr = pd.DataFrame(test_data, columns=['y_test', 'pred_logreg_test'])

confusion_matrix = pd.crosstab(con_matr['y_test'], con_matr['pred_logreg_test'], rownames=['True Value'], colnames=['Predicted'])

f,ax =plt.subplots(figsize=(10,8))
ax.set_title('Knn Accuracy Score')
sns.heatmap(confusion_matrix, annot=True)

plt.show()

# Model Tuning

In [ ]:
accuracies = []

for N in range(1,20):
    knn = KNeighborsClassifier(n_neighbors=N)

    knn.fit(x_train,y_train)
    y_pred = knn.predict(x_test)

    accu = accuracy_score(y_test, y_pred)
    accuracies.append(accu)

accuracies = np.array(accuracies)

f,ax =plt.subplots(figsize=(10,8))
ax.set_title('Knn Accuracy Score')
sns.lineplot(np.arange(1,20),accuracies)

In [ ]:
k_best = 1 + np.argmax(accuracies)
best_accu = np.max(accuracies)

kn_res = pd.DataFrame([k_best, best_accu]).transpose()
kn_res.columns = ['k_best', 'best_acu']
kn_res